## Ex 10.1

An introduction to Pytorch
A more tutorials on how to run pytorch can be found at: https://pytorch.org/tutorials/ 


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch import nn,tensor

## 1.1 PyTorch Tensors
https://pytorch.org/tutorials/beginner/basics/tensorqs_tutorial.html

The basic data type in PyTorch are tensors, they resemble Numpy Arrays, but are optimized to be used inside the Machine Learning Model. <br> 
In order to familiarize with tensors: <br>
1)Construct one tensor from a Python list <br>
2)Construct one tensor from a random numpy array <br>
3)Generate three tensors (one with random elements, one with all elemnts equal to 1, the third with all zeros) of shape (3,3) using the relative functions inside the module torch. <br>
4)Notice that tensors, like numpy arrays, support most of the standard mathematical operations, compute three different matematical operations (based on your choice) using the previously generated tensors <br>
5)Transform the results of your operations to numpy arrays.

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

## 1.2 DataSets
In Pytorch data are grouped in Datasets, to define a costum dataset, you need to derive a costum class from the torch.utils.data.Dataset class (see class inheritance in Python).<br>
A Dataset class must implement three functions: __init__, __len__, and __getitem__ , as in the following example:

https://pytorch.org/tutorials/beginner/basics/data_tutorial.html#creating-a-custom-dataset-for-your-files


    import os
    import pandas as pd
    from torchvision.io import read_image
    from torch.utils.data import Dataset
    
    class CustomImageDataset(Dataset):
        def __init__(self, annotations_file, img_dir, transform=None, target_transform=None):
            self.img_labels = pd.read_csv(annotations_file)
            self.img_dir = img_dir
            self.transform = transform
            self.target_transform = target_transform

        def __len__(self):
            return len(self.img_labels)

        def __getitem__(self, idx):
            img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
            image = read_image(img_path)
            label = self.img_labels.iloc[idx, 1]
            if self.transform:
                image = self.transform(image)
            if self.target_transform:
                label = self.target_transform(label)
            return image, label
In the example above the initializer saves a list of file names (img_labels) in the Dataset instance.
The \__getitem__(n) function reads from the hard drive the nth. 


1)Create two random tensors (X,Y) of length 200.<br>
2)Define a costum dataset which contains as objects two tensors (X and Y). The method \__getitem__(n) should return the nth values of these tensors\
3) Test if the code works

## 1.3 Neural Layers
Neural layers are function which apply transformations to tensors.
The linear transformation ($ y=x A^T +b$) is among the simplest:
https://pytorch.org/docs/stable/generated/torch.nn.Linear.html#torch.nn.Linear

1) create a linear layer which has both as input and output one element tensors. <br>
2) Find the parameters ($a$ and $b$) of the 1D transformation, trying to apply the transformation to different tensor (notice that the paramenter are randomly created every time you create a new layer)<br>
3) concatenate two layers in order to produce from a scalar imput an inter 3 elements tensor and a 9 element tensor in the end, 

## 1.4 Non linear activation functions
In the pytorch docs: https://pytorch.org/docs/stable/nn.html#non-linear-activations-weighted-sum-nonlinearity

One way to introduce non linearity in machine learning models is through the use of activation functions. 

1) Import from pytorch the Rectified Linear Unit function " ReLU "  and the hyperbolic tangent "tanh" function
2) Create a tensor of 100 evenly spaced points between -1 and 1
3) Plot the outputs of the two activation functions taking as argument the previously defined vector



In [ ]:
from torch import relu, tanh

## 1.5 Loss functions 
The way to measure the error of a Machine Learning prediction is through the use of one of the loss function. A list of the loss functions implemented in pytorch can be found here:
https://pytorch.org/docs/stable/nn.html#loss-functions

One of the most important loss functions for regression problems is the Mean Square Error (MSE loss function)

https://pytorch.org/docs/stable/generated/torch.nn.MSELoss.html#torch.nn.MSELoss

* Create two random tensors of length 4 
* Compute the mean squared difference of the two vectors using the pytorch's MSE Loss function 
* Compute the mean squared difference of the two vectors using standard matematical operations on tensors


# Example Neural Networks for Classification
Here we will provide a code snippet that classifies a dataset of handwritten digits. The layout of the code will provide you with insights and by adapting it, you will solve a classification problem in the next exercise.

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import numpy as np
import matplotlib.pyplot as plt

First, we will download the dataset and put it into datastructures provided by Pytorch. The transforms used here are specific to the dataset and transform the images into a `torch.tensor`, which is the datastructure used by pytorch (for an in depth tutorial on how Pytorch works and can be used for custom problems you can visit the [tutorials](https://pytorch.org/tutorials/)). In a second step the data points are normalized, which is a usual preprocessing step when training neural networks.

In [ ]:
# here we just create a directory "mnist" to store the dataset in
!mkdir mnist

In [ ]:
transform=transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.1307,), (0.3081,))])
dataset_train = datasets.MNIST('./mnist/', train=True, download=True,transform=transform)
dataset_test = datasets.MNIST('./mnist/', train=False,transform=transform)
train_loader = torch.utils.data.DataLoader(dataset_train, batch_size=16)
test_loader = torch.utils.data.DataLoader(dataset_test, batch_size=16)

Here you can have a look at the dataset to get a feeling on how it looks. The goal of the neural network is to use the image as an input and predict the number the image shows.

In [ ]:
plt.figure(figsize=(10,2),dpi=100)
for k,i in enumerate(np.random.randint(0, len(dataset_train),5)):
    plt.subplot(1,5,k+1)
    img, label = train_loader.dataset[i]
    img = img[0].numpy()
    print(label,img.shape)
    plt.imshow(img, cmap='gray')
    plt.title('Label = '+str(label))
    plt.axis('off')
plt.show()

For this prediction we will use a simple feedforward neural network architecture with one hidden layer and  ReLU-activation function. This activation function has excellent properties for training neural networks, since they solve the vanishing gradient problem and allow for deeper neural networks, but there are many different activation functions that can be used (see the [documentation](https://pytorch.org/docs/stable/nn.html) for a full list).
$$\mathrm{ReLU}(x) = max(0,x)$$
As output activation we use a logarithmic softmax, which can be seen as a mapping to the likelihoods of an image belonging to a certain class.

The architecture of our neural network consists of two fully connected layers defined in `__init__(self)`, which first map the $28*28=784$ pixels to 128 intermidiate values via matrix multiplication and an added bias term. This intermidiate vector is then mapped to a 10-dimensional ouput vector. The `forward(self, x)` method defines the forward pass of the neural network, where first the image is flattened into a vector, then the first fully connected layer is applied, then a ReLU activation, then the second fully connected layer and finally the logarithmic softmax function.

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(784, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output

After defining our architecture, we need to perform the training of the parameters. This optimization procedure is in most instances based on a gradient descent variety. Since we have a lot of training samples here, computing the full gradient of the loss function with respect to each adaptable weight is very expensive. Instead we opt for an approximation of the gradient of a minibatch of samples (e.g. 16 samples). This gives an approximate gradient direction, which allows us to update our weights and minimize our loss function more efficiently. Below you find a loop over our training and testing dataset. Read through the code and try to understand what each statement means and what happens.

Run the loop below and train the model (this might take a while).

In [ ]:
# number of iterations
epochs = 2
# step width in gradient-based optimization
learning_rate = 0.0001
# initialize our model with random parameters
model = Net()
# choose an optimizer
optimizer = optim.Adam(model.parameters(), lr = learning_rate)
# choose a loss function
loss_function = F.nll_loss

# initialize lists for plotting
learning_curve_train = []
learning_curve_test = []

for epoch in range(1, epochs+1):
    # temporary variables to track model convergence
    loss_train = []
    loss_test = []
    correct = 0
    
    # Training Loop
    for data, target in train_loader:
        # set gradient to zero
        optimizer.zero_grad()
        # predict with current model
        output = model(data)
        # compute loss of predictions and target values
        loss = loss_function(output, target)
        # Backpropagation of errors gives gradients
        loss.backward()
        # Update weights with approximate gradient
        optimizer.step()
        # Store loss
        loss_train.append(loss.item())

    # Stop tracking gradients for evaluation phase
    with torch.no_grad():
        # Test Loop
        for data, target in test_loader:
            # predict with current model
            output = model(data)
            # compute loss
            loss = loss_function(output, target)
            # store loss
            loss_test.append(loss.item())
            # transform output into predicted class
            pred = output.argmax(dim=1, keepdim=True)
            # compare predictions and labels and store 
            # number of correct predictions
            correct += pred.eq(target.view_as(pred)).sum().item()

    # Store loss at each iteration for plotting
    learning_curve_train.append(np.mean(loss_train))
    learning_curve_test.append(np.mean(loss_test))
    
    # Print to Stdout
    print('Epoch: {}\tTrain loss: {:.4f}\tTest loss: {:.4f}\tTest Accuracy: {}/{} ({:.0f}%)'.format(
        epoch, learning_curve_train[-1], learning_curve_test[-1], 
        correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

We can check if the model generalizes by looking at the learning curve (loss of training and testing data at each iteration).

In [ ]:
plt.plot(np.arange(1, epochs+1), learning_curve_train, label='Train')
plt.plot(np.arange(1, epochs+1), learning_curve_test, label='Test')
plt.xlabel('# Epochs')
plt.ylabel('Loss')
plt.legend()

Now we can look at some of the predictions.

In [ ]:
plt.figure(figsize=(10,2),dpi=100)
for k,i in enumerate(np.random.randint(0, len(dataset_test),5)):
    plt.subplot(1,5,k+1)
    img, label = test_loader.dataset[i]
    pred = torch.argmax(model(img)).item()
    img = img[0].numpy()
    plt.imshow(img, cmap='gray')
    plt.title('Prediction = '+str(pred))
    plt.xticks([])
    plt.yticks([])
    plt.xlabel('True = '+str(label))
plt.show()

In [ ]:
mim,Mim=(test_loader.dataset[7][0].numpy()).min() ,(test_loader.dataset[7][0].numpy()).max() 
mim,Mim,Mim-mim

# Ex 10.2

## Classification of Stars 
In this exercise we are going to adapt the previously provided code to work on a comparatively small dataset of star measurements from [Kaggle](https://www.kaggle.com/deepu1109/star-dataset). From just four measurements (Temperature, Luminosity, Radius, Absolute magnitude), we will try to predict the star type (Browm Dwarf, Red Dwarf, White Dwarf, Main Sequence, Supergiant, Hypergiant) by using a Feedforward Neural Network architecture that was trained on 200 measurements. By determining hyperparameters we will optimize our model before we will test the generalization of the model to not seen instances.

## 2.1
First load the dataset in `star_data.csv` from Moodle with pandas and familiarize yourself with the dataset. Extract the data into Numpy-arrays called `x` and `y` for input (the four measurments) and output (the star type) data respectively and make sure that the each array has the correct datatype (`np.float32` for `x` and `np.longlong` for `y`. **Hint**: you can use `.to_numpy()` on pandas dataframes and `.astype()` on numpy arrays to change the datatype.

## 2.2
Write a function `rescale(x)` that min-max-scales each of your input-values to values between 0 and 1 (see [here](https://towardsdatascience.com/everything-you-need-to-know-about-min-max-normalization-in-python-b79592732b79) for an introduction). 
$$x_{i,rescaled}=\frac{x_i-x_{i,min}}{x_{i,max}-x_{i,min}}$$

Split the pairs of inputs and outputs into 200 training samples and 40 testing samples. Shuffle your data beforehand to prevent unexpected behaviour. 

## 2.3
In the introductory example on the MNIST dataset, the dataset was already provided in a clean datastructure which allowed for convenient usage. Now we have to create our own dataset. For this we will use the `Dataset` class provided in Pytorch. Complete the `__len__(self)` and `__getitem__(self, idx)` methods to return the number of samples in the dataset and the sample (input and label) with index `idx`. Print the number of training- and test-datasets and output some of the input and output pairs by calling 5 random indices.

Using the `DataLoader` class, create a dataloader of your training- and test-data. You can specify a batch size for loading the data. This batch size can be seen as a hyperparameter, which allows you to tune the convergence of your model.

## 2.4
Adapt the previously provided `Net` class to work with the new data. For this change the architecture to reflect the new input and output dimensions of your data. You can also add additional layers and change the sizes of the hidden layers to tune model performance. **Hint:** Remove the `flatten()` function, since our data is already a vector and not a matrix. You can change the width of your hidden layers by changing the `out_features`-parameter in a `nn.Linear`-layer and the `in_features` of the following `nn.Linear`-layer. 

## 2.5
Use the provide code of the training and evaluation loop to the train your new model on the star data. Tune the hyperparameters (i.e., number of hidden layers, width of hidden layers, used activation functions, learning rate, number of epochs, training batch size) to optimize the training procedure and model performance and describe the influence of the different hyperparameters on the model performance and the training procedure.

## 2.6
Visualize the learning curves of training and test data by plotting the loss at each epoch against the number of epochs.

## 2.7
With your final model, predict the classes of the training and testing set one last time and visualize the corresponding confusion matrices. To do so, remember that the output of your model consists of a likelihood
for each class. Use the `argmax()` function to get the predicted class
label. To transform your torch tensors to python data you can use the
`.item()`-method. You can then use `sklearn.metrics.confusion_matrix()` (https://en.wikipedia.org/wiki/Confusion_matrix) to obtain
the confusion matrices, and `plt.imshow` to visualize them. 